In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import gc
gc.collect()

31

In [3]:
data = pd.read_csv(r'./input/lending-club-loan-data/loan_encoded.csv', low_memory=False)
data = data.drop(['Unnamed: 0'],axis=1)

In [4]:
data['Loan_Category_Bad'].value_counts()

0    1961647
1     299021
Name: Loan_Category_Bad, dtype: int64

In [5]:
#data1 = data.fillna(0)
#data1 = np.nan_to_num(data1)
#data = pd.DataFrame(data1, columns=data.columns)

In [6]:
data_good_loans = data.loc[data['Loan_Category_Bad']==0]
data_bad_loans = data.loc[data['Loan_Category_Bad']==1]

data_good_loans = data_good_loans.iloc[0:290000]
data_final = pd.concat([data_good_loans, data_bad_loans])

In [7]:
#start = 0
#end = 50000

# split files into 10 parts with each file having around 200k records
#for i in range(2):
#    data[start:end].to_csv(str('loan_encoded')+str(i)+str('.csv'))
#    start = end + 1
#    end = end + 200000

In [8]:
#data_final = data_final.drop(['tot_coll_amt'],axis=1)
#data_final = data_final.drop(['all_util','bc_util','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mort_acc','mths_since_recent_bc','mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl'],axis=1)
#data_final = data_final.drop(['pct_tl_nvr_dlq','percent_bc_gt_75'],axis=1)
#data_final = data_final.drop(['pct_tl_nvr_dlq','percent_bc_gt_75'],axis=1)
#data_final = np.where(data_final.values >= np.finfo(np.float64).max)

In [9]:
#type(data_final)

In [10]:
y = data_final['Loan_Category_Bad']
X = data_final.drop(['Loan_Category_Bad'],axis=1)


# Split data in test & train

In [11]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X,y)

# Baseline Model - Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(class_weight='balanced', solver='lbfgs')
lr.fit(trainX, trainY)
predictedY = lr.predict(testX)

C:\MachineLearning\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [13]:
from sklearn.metrics import confusion_matrix
confusion_matrix(testY, predictedY)

array([[72320,   541],
       [  867, 73528]], dtype=int64)

In [14]:
from sklearn.metrics import accuracy_score, recall_score, f1_score
print (accuracy_score(testY, predictedY))
print (recall_score(testY, predictedY))
print (f1_score(testY, predictedY))

0.9904384201662411
0.9883459909940184
0.9905162194201962


In [15]:
import pickle

# save model to file
pickle.dump(lr, open("lendingclub.pickle.dat", "wb"))

# Neural Model - Logistic Regression

In [16]:
#Convert X,y dataframes to np.array

X_list = X.values
y_list = np.array(y)

In [17]:
trainX, testX, trainY, testY = train_test_split(X_list,y_list)

In [18]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
#model.add(Dense(500, activation='relu'))
model.add(Dense(400, activation='relu'))
#model.add(Dense(200, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit (trainX,trainY, epochs=5, batch_size=64, validation_split=0.3)
model.summary()

Using TensorFlow backend.
W0923 14:35:50.199903 10448 deprecation_wrapper.py:119] From C:\MachineLearning\anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0923 14:35:50.323041 10448 deprecation_wrapper.py:119] From C:\MachineLearning\anaconda\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0923 14:35:50.340024 10448 deprecation_wrapper.py:119] From C:\MachineLearning\anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0923 14:35:50.343024 10448 deprecation_wrapper.py:119] From C:\MachineLearning\anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0923 14:35:50.371023 10448 deprecation_wr

Train on 309235 samples, validate on 132530 samples
Epoch 1/5
309235/309235 [==============================] - 10s 33us/step - loss: 7.8463 - acc: 0.5078 - val_loss: 7.8480 - val_acc: 0.5077
Epoch 2/5
309235/309235 [==============================] - 7s 23us/step - loss: 7.8463 - acc: 0.5078 - val_loss: 7.8480 - val_acc: 0.5077
Epoch 3/5
309235/309235 [==============================] - 7s 22us/step - loss: 7.8463 - acc: 0.5078 - val_loss: 7.8480 - val_acc: 0.5077
Epoch 4/5
309235/309235 [==============================] - 7s 23us/step - loss: 7.8463 - acc: 0.5078 - val_loss: 7.8480 - val_acc: 0.5077
Epoch 5/5
309235/309235 [==============================] - 7s 22us/step - loss: 7.8463 - acc: 0.5078 - val_loss: 7.8480 - val_acc: 0.5077
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 400)               52400     
___________________________________________________________

In [19]:
predictedY = model.predict(testX)

print (confusion_matrix(testY, predictedY))
print (accuracy_score(testY, predictedY))
print (recall_score(testY, predictedY))
print (f1_score(testY, predictedY))

[[    0 72565]
 [    0 74691]]
0.5072187211386973
1.0
0.6730525756148992
